# Super hacky and slow regex search engine

This is just another way for me to understand what's on the data. It's uses regular expressions to browse through the papers and display sorrounding snippets on markdown

In [1]:
from pathlib import Path

DATA_PATH = Path('/kaggle/input/CORD-19-research-challenge/2020-03-13/')
DATA_PATH = Path('~/Downloads/2020-03-13/').expanduser()

In [2]:
from tqdm import tqdm
import json

def iter_papers():
    """
    Iterate over all directories and yields all papers
    """
    dirs = 'comm_use_subset noncomm_use_subset pmc_custom_license biorxiv_medrxiv'.split()
    for dir in dirs:
        fnames = (DATA_PATH / dir / dir).glob('*')
        for fname in fnames:
            with fname.open() as f:
                content = json.load(f)
            yield content    

In [29]:
import re
from collections import namedtuple

Window = namedtuple('Window', ['field', 'left', 'center', 'right'])

def _get_win_pat(name, size):
    return '(?P<%s>\w+(\W+\w+){,%s})' % (name, size)

def iter_fields(paper):
    def get(k):
        res = paper
        for kk in k.split('.'): res=res.get(kk)
        return res
        
    for key in 'abstract body_text metadata.title'.split():
        val = get(key)
        if isinstance(val, list): val = '\n\n'.join([e['text'] for e in val])
        
        yield key, val

def iter_gd(paper, pat_str, left_wsize=5, right_wsize=5):
    """
    Generator for matching snippets
    """
    pat_str = pat_str.replace(' ', '\W+')
    full_pat_str = f"{_get_win_pat('left', left_wsize)} (?P<center>{pat_str}) {_get_win_pat('right', left_wsize)}"
    pat = re.compile(full_pat_str, re.I | re.M)
    
    for key, field in iter_fields(paper):
        for match in pat.finditer(field):
            yield Window(field=key, **match.groupdict())


# For this first attempt, just limit the papers to those saying either covid or coronavirus on the title

In [71]:
it = iter_papers()

from tqdm.auto import tqdm

inserted = tqdm(desc='inserted')
covid_papers = []
all_papers = []
pat = re.compile('covid|coronavirus', re.I)
all_papers = list(tqdm(iter_papers()))

for p in tqdm(all_papers):
    all_papers.append(p)
    if pat.search(p['metadata']['title']): 
        covid_papers.append(p)
        inserted.update()

In [40]:
from IPython.display import display, Markdown

dm = lambda x: display(Markdown(x))

pat_matches = []
pat_str = (
    'transmission|incubation|environmental stability|nasal discharge|sputum|urine|fecal matter|blood|'
    'virus persistence|diagnosis|diagnostic|phenotypic|immune response|immunity'
)
for paper in tqdm(all_papers):
    pat_matches.extend(
        iter_gd(
            paper, pat_str, left_wsize=30, right_wsize=30
        )
    )
    

In [41]:
import pickle as pkl

# with open('all_matches.pkl', 'wb') as f:
#     pkl.dump(pat_matches, f, 4)

In [56]:
matches_distr = defaultdict(lambda: defaultdict(int))

for match in pat_matches:
    for tok in tokenize(match.left + ' ' + match.right):
        matches_distr[' '.join(tokenize(match.center.lower()))][tok] += 1

In [48]:
from tqdm.auto import tqdm
from collections import defaultdict
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

pat = re.compile('\w+')

def tokenize(txt):
    return [tok for tok in pat.findall(txt.lower()) if tok not in stopwords]

default_distr = defaultdict(int)
for paper in tqdm(relevant):
    for _, field in iter_fields(paper):
        for tok in tokenize(field):
            default_distr[tok] += 1
    

In [57]:
matches_distr.keys()

dict_keys(['diagnostic', 'incubation', 'immunity', 'nasal discharge', 'immune response', 'diagnosis', 'blood', 'transmission', 'phenotypic', 'sputum', 'virus persistence', 'urine', 'fecal matter', 'environmental stability'])

In [94]:
relevance = {tok: cnt / default_distr[tok] for tok, cnt in matches_distr['transmission'].items() if cnt > 20}
relevant_keywords = sorted(relevance.items(), key=lambda x: -x[1])[:30]
relevant_keywords

[('humanto', 0.6710526315789473),
 ('tohuman', 0.6131386861313869),
 ('agmps', 0.5671641791044776),
 ('crossspecies', 0.5272727272727272),
 ('mdrgn', 0.48148148148148145),
 ('fomite', 0.46779661016949153),
 ('interspecies', 0.4674457429048414),
 ('stuttering', 0.45132743362831856),
 ('cauchemez', 0.44285714285714284),
 ('jem', 0.41836734693877553),
 ('onward', 0.4106583072100313),
 ('ultramicrotome', 0.4074074074074074),
 ('jeol', 0.4),
 ('outbreaker', 0.4),
 ('fomites', 0.3730964467005076),
 ('subcritical', 0.37),
 ('perv', 0.3620689655172414),
 ('superspreading', 0.34980988593155893),
 ('multihost', 0.3492063492063492),
 ('spreaders', 0.34296028880866425),
 ('presymptomatic', 0.3287671232876712),
 ('uranyl', 0.3253731343283582),
 ('hitachi', 0.3229166666666667),
 ('superspreaders', 0.3221476510067114),
 ('phosphotungstic', 0.313953488372093),
 ('airborne', 0.31272563176895307),
 ('tecnai', 0.30701754385964913),
 ('gatan', 0.3068181818181818),
 ('aphid', 0.3006993006993007),
 ('ird', 

In [119]:
transmission_matches = []
for paper in tqdm(covid_papers):
    for match in iter_gd(paper, 'transmission', 60, 60):
        transmission_matches.append((paper, match))

In [107]:
from collections import defaultdict
from IPython.display import display, Markdown

dm = lambda x: display(Markdown(x))

d = defaultdict(list)

patterns = [(k, re.compile(f'(\W|^)({k})(\W|$)', re.I)) for k, _ in relevant_keywords]

for match in tqdm(transmission_matches):
    txt = match.left + ' ' + match.center + ' ' + match.right
    for k, p in patterns:
        if p.search(txt): 
            d[k].append(match)


In [110]:
d['interspecies'][0]

Window(field='abstract', left='NL63] and ß-CoV [including HCoV-OC43, HKU1, Severe Acute Respiratory Syndrome CoV (SARS-CoV) and Middle East Respiratory Syndrome CoV (MERS-CoV)] are known to infect human beings 1 .\n\nInterspecies', center='transmission', right='is a common phenomenon for CoVs that may be responsible for generation of new CoV epidemics during viral evolution 2-5 . For instance, the feline CoV (FCoV) type II, a member')

In [111]:
re.sub?

In [118]:
def index(string, pat):
    try:
        return string.lower().index(pat)
    except:
        return -1

def highlight(txt, pat):
    return re.sub(f'({pat})', '**\g<0>**', txt, flags=re.I)
    
for k, l in sorted(d.items(), key=lambda x: -len(x[1])):
    dm(f'## {k}')
    for match in l:
        lindex = max(50, len(match.left) - 1 - index(match.left, k))
        rindex = max(50, index(match.right, k))
        dm(f"{highlight(match.left, k)} **{match.center}** {highlight(match.right, k)}")
        dm('---')



## interspecies

NL63] and ß-CoV [including HCoV-OC43, HKU1, Severe Acute Respiratory Syndrome CoV (SARS-CoV) and Middle East Respiratory Syndrome CoV (MERS-CoV)] are known to infect human beings 1 .

**Interspecies** **transmission** is a common phenomenon for CoVs that may be responsible for generation of new CoV epidemics during viral evolution 2-5 . For instance, the feline CoV (FCoV) type II, a member

---

outbreaks that continuously threaten human health 2,3 . This phenomenon is exemplified by the recent emergence of SARS-CoV and MERS-CoV 6,7 . However, the detailed evolutionary mechanism of **interspecies** **transmission** and the persistence of CoVs in specific hosts have yet to be fully elucidated

---

...

DI water, placed on 400-mesh, carbon-stabilized grids and negatively stained for 30-60 seconds with 2% phosphotungstic acid solution. The grids were examined on a **JEOL** JEM-100CX II **transmission** electron microscope and evaluated by an experienced electron microscopist.

Data were analyzed using commercial software (SigmaPlot; SPSS, Chicago, Illinois). Normality of the data and equality of variances were assessed using the

---

## subcritical

study (61); whereas, an exponential growth pattern was estimated for Hubei (p~1.0) (61).

A previous study on the 2015 MERS outbreak in South Korea reported substantial potential for superspreading **transmission** despite a **subcritical** Rt (62) . The lower estimate of the dispersion parameter in our study also indicate significant transmission heterogeneity in Singapore. Superspreading events of MERS-CoV and SARS-CoV associated

---

In [89]:
match.right.index(k)

ValueError: substring not found